# Combining Unstructured and Structured data

The dataset for our proposed MVP consists only of structured data. Once we undestood how to query structured data we attempted to add unstructured documents such as pdf`s to our chatbot. A promising way to achieve that was a tool developed by llama-index called Auto SQL Query Engine.

## SQL Auto Vector Query Engine

This query engine allows you to combine insights from your structured tables with your unstructured data. It first uses LLM to reason weather the question shuld be answerd by looking at structured or unstructured pasrt of the dataset. Once it reaches a decision it then utilizes one of the available tools 

 - SQLAutoVectorQueryEngine,
 - RetrieverQueryEngine

Since our dataset was initialy in a csv form, we developed a normalized postgress databse for our data. In order to run the code below you will need to setup a local database and use the provided scripts to create the neccessary tables **cdc_session.sql** and fill the databse with data contained in the dataset **transfer_data_to_database.py**. You can find the files in the Scripts folder. 

Your username, database name, passwords and api keys should all go into the .env file. Once you have that setup, you can run this notebook.

In [2]:
import os
import openai
from dotenv import load_dotenv

# enter you credentials and api key into the .env file and load them
load_dotenv('../.env', override=True)

openai.api_key = os.getenv('OPENAI_API_KEY')

# get Database credentials from the .env file
load_dotenv("../.env", override=True)
DBUSER = os.environ["DBUSER"]
DBPASS = os.environ["DBPASS"]
DBHOST = os.environ["DBHOST"]
DBNAME = os.environ["DBNAME"]
DATABASE_URI = f"postgresql://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}"
if DBHOST != "localhost":
    DATABASE_URI += "?sslmode=require"

#### Create Common Objects

These are common objects for the SQL and Vectorized query engines. This setup uses a free version of pinecone index.

In [3]:
# define pinecone index
import pinecone
import os

# enter your PINECONE api key into .env file
api_key = os.getenv('PINECONE_API_KEY')
pinecone.init(api_key=api_key, environment='gcp-starter')
print(api_key)
# dimensions are for text-embedding-ada-002
# pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")
pinecone_index = pinecone.Index("quickstart")

pinecone_index.delete('quickstart')


from llama_index.node_parser.simple import SimpleNodeParser
from llama_index import ServiceContext, VectorStoreIndex, SQLDatabase
from llama_index.storage import StorageContext
from llama_index.vector_stores import PineconeVectorStore
from llama_index.text_splitter import TokenTextSplitter
from llama_index.llms import OpenAI

# define node parser and LLM
chunk_size = 1024
llm = OpenAI(temperature=0, model="gpt-4", streaming=True)
service_context = ServiceContext.from_defaults(chunk_size=chunk_size, llm=llm)
text_splitter = TokenTextSplitter(chunk_size=chunk_size)
node_parser = SimpleNodeParser.from_defaults(text_splitter=text_splitter)

# define pinecone vector index
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex([], storage_context=storage_context)

/home/adnan/MADS/capstone/chatbot/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


03005a3c-aca8-4caf-aa57-bfbf4e1687a8


## Setup

In [4]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Connect to database

In [5]:
%load_ext sql
from sqlalchemy import (
    create_engine,
    MetaData
)

In [6]:
engine = create_engine(DATABASE_URI)

In [7]:
metadata_obj = MetaData()

# Reflect the current state of the database into the metadata
metadata_obj.reflect(bind=engine)

# Print table names
print(metadata_obj.tables.keys())

dict_keys(['surveydata', 'datasource', 'datatypes', 'locations', 'questions', 'topics', 'stratifications', 'stratificationcategories'])


### Build SQL Index 

In [8]:
sql_database = SQLDatabase(engine, include_tables=["datasource", "datatypes", "locations", "questions", "stratificationcategories", "stratifications", "topics", "surveydata"])

In [9]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

In [30]:
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    # tables=["chronic_disease_indicators"],
)

In [42]:
response_template = """
## Question

{question}

## Answer
```
{response}
```
## Generated SQL Query
```
{sql}
```
"""

In [12]:
from IPython.display import Markdown, display
def chat_to_sql(question: str or list[str],tables: list[str] or None = None,synthesize_response: bool = True,):
    query_engine = NLSQLTableQueryEngine(
        sql_database=sql_database,
        tables=tables,
        synthesize_response=synthesize_response,
        service_context=service_context,
        )
    
    try:
        response = query_engine.query(question)
        response_md = str(response)
        sql = response.metadata["sql_query"]
    except Exception as ex:
        response_md = "Error"
        sql = f"ERROR: {str(ex)}"

   
    display(Markdown(response_template.format(
        question=question,
        response=response_md,
        sql=sql,
    )))

In [13]:
chat_to_sql("What are some of the topics tracked in the dataset?")

INFO:llama_index.indices.struct_store.sql_retriever:> Table desc str: Table 'datasource' has columns: datasourceid (INTEGER), datasource (VARCHAR(255)), and foreign keys: .

Table 'datatypes' has columns: datatypeid (INTEGER), datavaluetype (VARCHAR(100)), datavalueunit (VARCHAR(100)), and foreign keys: .

Table 'locations' has columns: locationid (INTEGER), locationabbr (VARCHAR(10)), locationdesc (VARCHAR(255)), geolocation (VARCHAR(255)), and foreign keys: .

Table 'questions' has columns: questionid (INTEGER), question (VARCHAR(500)), topicid (INTEGER), and foreign keys: ['topicid'] -> topics.['topicid'].

Table 'stratificationcategories' has columns: stratificationcategoryid (INTEGER), stratificationcategory (VARCHAR(255)), and foreign keys: .

Table 'stratifications' has columns: stratificationid (INTEGER), stratification (VARCHAR(255)), stratificationcategoryid (INTEGER), and foreign keys: ['stratificationcategoryid'] -> stratificationcategories.['stratificationcategoryid'].

Ta


## Question

What are some of the topics tracked in the dataset?

## Answer
```
Some of the topics tracked in the dataset include Asthma, Cancer, Chronic Kidney Disease, Chronic Obstructive Pulmonary Disease, Cardiovascular Disease, Diabetes, Disability, Reproductive Health, Tobacco, and Alcohol.
```
## Generated SQL Query
```
SELECT topic FROM topics LIMIT 10;
```


In [14]:
chat_to_sql('List Cancer related indicators available in the dataset')


INFO:llama_index.indices.struct_store.sql_retriever:> Table desc str: Table 'datasource' has columns: datasourceid (INTEGER), datasource (VARCHAR(255)), and foreign keys: .

Table 'datatypes' has columns: datatypeid (INTEGER), datavaluetype (VARCHAR(100)), datavalueunit (VARCHAR(100)), and foreign keys: .

Table 'locations' has columns: locationid (INTEGER), locationabbr (VARCHAR(10)), locationdesc (VARCHAR(255)), geolocation (VARCHAR(255)), and foreign keys: .

Table 'questions' has columns: questionid (INTEGER), question (VARCHAR(500)), topicid (INTEGER), and foreign keys: ['topicid'] -> topics.['topicid'].

Table 'stratificationcategories' has columns: stratificationcategoryid (INTEGER), stratificationcategory (VARCHAR(255)), and foreign keys: .

Table 'stratifications' has columns: stratificationid (INTEGER), stratification (VARCHAR(255)), stratificationcategoryid (INTEGER), and foreign keys: ['stratificationcategoryid'] -> stratificationcategories.['stratificationcategoryid'].

Ta


## Question

List Cancer related indicators available in the dataset

## Answer
```
There are no cancer-related indicators available in the dataset.
```
## Generated SQL Query
```
SELECT DISTINCT questions.question 
FROM questions 
JOIN topics ON questions.topicid = topics.topicid 
WHERE topics.topic LIKE '%Cancer%'
```


In [15]:
chat_to_sql( """ List Cancer related Indicators for which there are data for the last year of any available data.
    """)


INFO:llama_index.indices.struct_store.sql_retriever:> Table desc str: Table 'datasource' has columns: datasourceid (INTEGER), datasource (VARCHAR(255)), and foreign keys: .

Table 'datatypes' has columns: datatypeid (INTEGER), datavaluetype (VARCHAR(100)), datavalueunit (VARCHAR(100)), and foreign keys: .

Table 'locations' has columns: locationid (INTEGER), locationabbr (VARCHAR(10)), locationdesc (VARCHAR(255)), geolocation (VARCHAR(255)), and foreign keys: .

Table 'questions' has columns: questionid (INTEGER), question (VARCHAR(500)), topicid (INTEGER), and foreign keys: ['topicid'] -> topics.['topicid'].

Table 'stratificationcategories' has columns: stratificationcategoryid (INTEGER), stratificationcategory (VARCHAR(255)), and foreign keys: .

Table 'stratifications' has columns: stratificationid (INTEGER), stratification (VARCHAR(255)), stratificationcategoryid (INTEGER), and foreign keys: ['stratificationcategoryid'] -> stratificationcategories.['stratificationcategoryid'].

Ta


## Question

 List Cancer related Indicators for which there are data for the last year of any available data.
    

## Answer
```
There are no cancer-related indicators for which data is available for the most recent year.
```
## Generated SQL Query
```
SELECT DISTINCT questions.question 
FROM questions 
JOIN surveydata ON questions.questionid = surveydata.questionid 
JOIN topics ON questions.topicid = topics.topicid 
WHERE topics.topic LIKE '%Cancer%' 
AND surveydata.yearend = (SELECT MAX(yearend) FROM surveydata);
```


In [16]:
chat_to_sql( "How many diferent datatypes are in the dataset? I am interested in datatypes such as percent, Number etc..")

INFO:llama_index.indices.struct_store.sql_retriever:> Table desc str: Table 'datasource' has columns: datasourceid (INTEGER), datasource (VARCHAR(255)), and foreign keys: .

Table 'datatypes' has columns: datatypeid (INTEGER), datavaluetype (VARCHAR(100)), datavalueunit (VARCHAR(100)), and foreign keys: .

Table 'locations' has columns: locationid (INTEGER), locationabbr (VARCHAR(10)), locationdesc (VARCHAR(255)), geolocation (VARCHAR(255)), and foreign keys: .

Table 'questions' has columns: questionid (INTEGER), question (VARCHAR(500)), topicid (INTEGER), and foreign keys: ['topicid'] -> topics.['topicid'].

Table 'stratificationcategories' has columns: stratificationcategoryid (INTEGER), stratificationcategory (VARCHAR(255)), and foreign keys: .

Table 'stratifications' has columns: stratificationid (INTEGER), stratification (VARCHAR(255)), stratificationcategoryid (INTEGER), and foreign keys: ['stratificationcategoryid'] -> stratificationcategories.['stratificationcategoryid'].

Ta


## Question

How many diferent datatypes are in the dataset? I am interested in datatypes such as percent, Number etc..

## Answer
```
There are 19 different data types in the dataset.
```
## Generated SQL Query
```
SELECT COUNT(DISTINCT datavaluetype) FROM datatypes;
```


In [23]:
question = """ What is the mean value for indicator "Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50-75 years" in the last year of available data? Provide answer for Overall stratification group.
    """


chat_to_sql(question)

INFO:llama_index.indices.struct_store.sql_retriever:> Table desc str: Table 'datasource' has columns: datasourceid (INTEGER), datasource (VARCHAR(255)), and foreign keys: .

Table 'datatypes' has columns: datatypeid (INTEGER), datavaluetype (VARCHAR(100)), datavalueunit (VARCHAR(100)), and foreign keys: .

Table 'locations' has columns: locationid (INTEGER), locationabbr (VARCHAR(10)), locationdesc (VARCHAR(255)), geolocation (VARCHAR(255)), and foreign keys: .

Table 'questions' has columns: questionid (INTEGER), question (VARCHAR(500)), topicid (INTEGER), and foreign keys: ['topicid'] -> topics.['topicid'].

Table 'stratificationcategories' has columns: stratificationcategoryid (INTEGER), stratificationcategory (VARCHAR(255)), and foreign keys: .

Table 'stratifications' has columns: stratificationid (INTEGER), stratification (VARCHAR(255)), stratificationcategoryid (INTEGER), and foreign keys: ['stratificationcategoryid'] -> stratificationcategories.['stratificationcategoryid'].

Ta


## Question

 What is the mean value for indicator "Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50-75 years" in the last year of available data? Provide answer for Overall stratification group.
    

## Answer
```
The data for the mean value of the indicator "Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50-75 years" for the 'Overall' stratification group in the last year of available data is not available.
```
## Generated SQL Query
```
SELECT AVG(surveydata.datavaluealt) AS mean_value
FROM surveydata
JOIN questions ON surveydata.questionid = questions.questionid
JOIN stratifications ON surveydata.stratificationid = stratifications.stratificationid
WHERE questions.question = 'Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50-75 years'
AND stratifications.stratification = 'Overall'
AND surveydata.yearend = (SELECT MAX(yearend) FROM surveydata);
```


In [24]:
question = """ What is the value for "Cancer of the female breast, mortality" in Wisconsin. Provide answer for Overall stratification group.
    """


chat_to_sql(question)

INFO:llama_index.indices.struct_store.sql_retriever:> Table desc str: Table 'datasource' has columns: datasourceid (INTEGER), datasource (VARCHAR(255)), and foreign keys: .

Table 'datatypes' has columns: datatypeid (INTEGER), datavaluetype (VARCHAR(100)), datavalueunit (VARCHAR(100)), and foreign keys: .

Table 'locations' has columns: locationid (INTEGER), locationabbr (VARCHAR(10)), locationdesc (VARCHAR(255)), geolocation (VARCHAR(255)), and foreign keys: .

Table 'questions' has columns: questionid (INTEGER), question (VARCHAR(500)), topicid (INTEGER), and foreign keys: ['topicid'] -> topics.['topicid'].

Table 'stratificationcategories' has columns: stratificationcategoryid (INTEGER), stratificationcategory (VARCHAR(255)), and foreign keys: .

Table 'stratifications' has columns: stratificationid (INTEGER), stratification (VARCHAR(255)), stratificationcategoryid (INTEGER), and foreign keys: ['stratificationcategoryid'] -> stratificationcategories.['stratificationcategoryid'].

Ta


## Question

 What is the value for "Cancer of the female breast, mortality" in Wisconsin. Provide answer for Overall stratification group.
    

## Answer
```
There is no available data for "Cancer of the female breast, mortality" in the overall stratification group for Wisconsin.
```
## Generated SQL Query
```
SELECT surveydata.datavalue, surveydata.yearstart, surveydata.yearend
FROM surveydata
JOIN questions ON surveydata.questionid = questions.questionid
JOIN locations ON surveydata.locationid = locations.locationid
JOIN stratifications ON surveydata.stratificationid = stratifications.stratificationid
WHERE questions.question = 'Cancer of the female breast, mortality'
AND locations.locationdesc = 'Wisconsin'
AND stratifications.stratification = 'Overall'
ORDER BY surveydata.yearstart DESC;
```


In [25]:
question = 'Which state has the worst values for "Cancer of the female breast, mortality" for the last year of available data for that particular indicator. Where does it rank in Poverty indicator among states? Provide answer for Overall stratification group.'

chat_to_sql(question)

INFO:llama_index.indices.struct_store.sql_retriever:> Table desc str: Table 'datasource' has columns: datasourceid (INTEGER), datasource (VARCHAR(255)), and foreign keys: .

Table 'datatypes' has columns: datatypeid (INTEGER), datavaluetype (VARCHAR(100)), datavalueunit (VARCHAR(100)), and foreign keys: .

Table 'locations' has columns: locationid (INTEGER), locationabbr (VARCHAR(10)), locationdesc (VARCHAR(255)), geolocation (VARCHAR(255)), and foreign keys: .

Table 'questions' has columns: questionid (INTEGER), question (VARCHAR(500)), topicid (INTEGER), and foreign keys: ['topicid'] -> topics.['topicid'].

Table 'stratificationcategories' has columns: stratificationcategoryid (INTEGER), stratificationcategory (VARCHAR(255)), and foreign keys: .

Table 'stratifications' has columns: stratificationid (INTEGER), stratification (VARCHAR(255)), stratificationcategoryid (INTEGER), and foreign keys: ['stratificationcategoryid'] -> stratificationcategories.['stratificationcategoryid'].

Ta


## Question

Which state has the worst values for "Cancer of the female breast, mortality" for the last year of available data for that particular indicator. Where does it rank in Poverty indicator among states? Provide answer for Overall stratification group.

## Answer
```
I'm sorry, but the SQL query provided is invalid and therefore I cannot provide the requested information.
```
## Generated SQL Query
```
SELECT COUNT(*)+1 AS rank 
FROM (
    SELECT surveydata.datavalue 
    FROM surveydata 
    JOIN questions ON surveydata.questionid = questions.questionid 
    JOIN locations ON surveydata.locationid = locations.locationid 
    JOIN stratifications ON surveydata.stratificationid = stratifications.stratificationid 
    WHERE questions.question = 'Poverty' 
    AND stratifications.stratification = 'Overall' 
    AND surveydata.yearstart = 2019 
    AND locations.locationdesc != 'Mississippi' 
    ORDER BY surveydata.datavalue DESC
) AS subquery 
WHERE datavalue > 30.5;
```


In [26]:
question = """How does "Hospitalizations for Asthma" indicator corelate with "Sale of cigarette packs" Provide answer for Overall stratification group.
"""
chat_to_sql(question)

INFO:llama_index.indices.struct_store.sql_retriever:> Table desc str: Table 'datasource' has columns: datasourceid (INTEGER), datasource (VARCHAR(255)), and foreign keys: .

Table 'datatypes' has columns: datatypeid (INTEGER), datavaluetype (VARCHAR(100)), datavalueunit (VARCHAR(100)), and foreign keys: .

Table 'locations' has columns: locationid (INTEGER), locationabbr (VARCHAR(10)), locationdesc (VARCHAR(255)), geolocation (VARCHAR(255)), and foreign keys: .

Table 'questions' has columns: questionid (INTEGER), question (VARCHAR(500)), topicid (INTEGER), and foreign keys: ['topicid'] -> topics.['topicid'].

Table 'stratificationcategories' has columns: stratificationcategoryid (INTEGER), stratificationcategory (VARCHAR(255)), and foreign keys: .

Table 'stratifications' has columns: stratificationid (INTEGER), stratification (VARCHAR(255)), stratificationcategoryid (INTEGER), and foreign keys: ['stratificationcategoryid'] -> stratificationcategories.['stratificationcategoryid'].

Ta


## Question

How does "Hospitalizations for Asthma" indicator corelate with "Sale of cigarette packs" Provide answer for Overall stratification group.


## Answer
```
The data does not provide any correlation between the "Hospitalizations for Asthma" indicator and the "Sale of cigarette packs" for the Overall stratification group.
```
## Generated SQL Query
```
SELECT surveydata.datavalue, questions.question, stratifications.stratification
FROM surveydata
JOIN questions ON surveydata.questionid = questions.questionid
JOIN stratifications ON surveydata.stratificationid = stratifications.stratificationid
WHERE questions.question IN ('Hospitalizations for Asthma', 'Sale of cigarette packs') AND stratifications.stratification = 'Overall'
ORDER BY surveydata.datavalue DESC;
```


In [27]:
question = """Find the pair of Indicators with the highest correlation
"""

chat_to_sql(question)

INFO:llama_index.indices.struct_store.sql_retriever:> Table desc str: Table 'datasource' has columns: datasourceid (INTEGER), datasource (VARCHAR(255)), and foreign keys: .

Table 'datatypes' has columns: datatypeid (INTEGER), datavaluetype (VARCHAR(100)), datavalueunit (VARCHAR(100)), and foreign keys: .

Table 'locations' has columns: locationid (INTEGER), locationabbr (VARCHAR(10)), locationdesc (VARCHAR(255)), geolocation (VARCHAR(255)), and foreign keys: .

Table 'questions' has columns: questionid (INTEGER), question (VARCHAR(500)), topicid (INTEGER), and foreign keys: ['topicid'] -> topics.['topicid'].

Table 'stratificationcategories' has columns: stratificationcategoryid (INTEGER), stratificationcategory (VARCHAR(255)), and foreign keys: .

Table 'stratifications' has columns: stratificationid (INTEGER), stratification (VARCHAR(255)), stratificationcategoryid (INTEGER), and foreign keys: ['stratificationcategoryid'] -> stratificationcategories.['stratificationcategoryid'].

Ta


## Question

Find the pair of Indicators with the highest correlation


## Answer
```
I'm sorry, but the SQL statement provided is invalid and cannot be executed. Please check the syntax and try again.
```
## Generated SQL Query
```
SELECT s1.datavalue AS datavalue1, s2.datavalue AS datavalue2, corr(s1.datavaluealt, s2.datavaluealt) AS correlation
FROM surveydata s1
JOIN surveydata s2 ON s1.yearstart = s2.yearstart AND s1.yearend = s2.yearend AND s1.locationid = s2.locationid
WHERE s1.questionid <> s2.questionid
ORDER BY correlation DESC
LIMIT 1;
```


In [24]:
###########################################################################################################################################

### Build Vector Index

In [20]:
from llmsherpa.readers import LayoutPDFReader

llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_url = "https://www.cdc.gov/mmwr/pdf/rr/rr6401.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(pdf_url)

In [22]:
from llama_index.readers.schema.base import Document
from llama_index import VectorStoreIndex

vector_index = VectorStoreIndex([])
for chunk in doc.chunks():
    vector_index.insert(Document(text=chunk.to_context_text(), extra_info={}))
query_engine = vector_index.as_query_engine()

# Let's run one query
# response = query_engine.query("tell me about background and rationale for cdc`s chronic disease indicators?")
# print(response)

Lets query the pdf document and see if we can get any answers at all to check if the code is working.

In [23]:
response = query_engine.query("Tell me something about the rationale behind chronic disease indicators.")
print(response)

The rationale behind chronic disease indicators is to provide a comprehensive set of surveillance indicators that can be used by public health professionals and policymakers to retrieve data for chronic diseases and risk factors. These indicators are essential for surveillance, prioritization, and evaluation of public health interventions for chronic disease. The chronic disease indicators were established to address the important public health problem of chronic diseases, which can result in morbidity, mortality, disability, and decreased quality of life. The indicators are developed through collaboration among CDC, the Council of State and Territorial Epidemiologists, and the National Association of Chronic Disease Directors, and are based on a comprehensive review to ensure they capture high-impact diseases and conditions as well as emerging topics.


## Putting it all together

In [25]:
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.query_engine import SQLAutoVectorQueryEngine
from llama_index.indices.vector_store.retrievers import VectorIndexAutoRetriever

from llama_index.vector_stores.types import MetadataInfo, VectorStoreInfo
from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine

In [31]:



vector_store_info = VectorStoreInfo(
    content_info="report from 2015 about the program of gathering chronic disease indiocators and state of those inidcators across the united states",
    metadata_info=[
        MetadataInfo(
            name="chronic disease indicator", type="str", description=""
        ),
    ],
)
vector_auto_retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info
)

retriever_query_engine = RetrieverQueryEngine.from_args(
    vector_auto_retriever, service_context=service_context
)

In [32]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " a table containing: chronic disease indicator data"
    ),
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=retriever_query_engine,
    description=(
        f"Useful for answering semantic questions about cdc chronic disease indicators"
    ),
)

### Define SQLAutoVectorQueryEngine and ask Questions

In [33]:
query_engine = SQLAutoVectorQueryEngine(
    sql_tool, vector_tool, service_context=service_context
)

In [39]:
response = query_engine.query(
    'List 5 Cancer indicators in the dataset?'
)

Querying SQL database: The first choice seems more relevant as it involves translating a natural language query into a SQL query over a table containing chronic disease indicator data. The question 'List 5 Cancer indicators in the dataset?' could be considered a natural language query that needs to be translated into a SQL query to retrieve the required data from a dataset.
INFO:llama_index.query_engine.sql_join_query_engine:> Querying SQL database: The first choice seems more relevant as it involves translating a natural language query into a SQL query over a table containing chronic disease indicator data. The question 'List 5 Cancer indicators in the dataset?' could be considered a natural language query that needs to be translated into a SQL query to retrieve the required data from a dataset.
> Querying SQL database: The first choice seems more relevant as it involves translating a natural language query into a SQL query over a table containing chronic disease indicator data. The q

In [ ]:
response = query_engine.query(
    'Give me max values for question Pneumococcal vaccination among noninstitutionalized adults aged 18-64 years with diagnosed diabetes. Group by  locationdesc an avalilable data value types. Make sure you omitt null values, text and cast to numeric. Format output as a tabele'
)

In [40]:
response = query_engine.query(
    'which state has the highets value for indicator mentioning hospitalization in topic Diabetes?'
)

Querying SQL database: The question is asking for specific data (highest value for a disease indicator) from a table, which suggests it needs a SQL query to retrieve this information. Choice 1 is about translating a natural language query into a SQL query over a table containing chronic disease indicator data, which aligns with the requirements of the question.
INFO:llama_index.query_engine.sql_join_query_engine:> Querying SQL database: The question is asking for specific data (highest value for a disease indicator) from a table, which suggests it needs a SQL query to retrieve this information. Choice 1 is about translating a natural language query into a SQL query over a table containing chronic disease indicator data, which aligns with the requirements of the question.
> Querying SQL database: The question is asking for specific data (highest value for a disease indicator) from a table, which suggests it needs a SQL query to retrieve this information. Choice 1 is about translating a 

In [41]:
response = query_engine.query(
    'which state has the highets value for indicators mentioning mortality in topic Cancer. Remeber to omit null values and cast datavalue to numeric '
)

Querying SQL database: The question is asking for a specific data query, which involves translating a natural language query into a SQL query. This is more closely related to the first choice, which mentions translating a natural language query into a SQL query over a table containing chronic disease indicator data.
INFO:llama_index.query_engine.sql_join_query_engine:> Querying SQL database: The question is asking for a specific data query, which involves translating a natural language query into a SQL query. This is more closely related to the first choice, which mentions translating a natural language query into a SQL query over a table containing chronic disease indicator data.
> Querying SQL database: The question is asking for a specific data query, which involves translating a natural language query into a SQL query. This is more closely related to the first choice, which mentions translating a natural language query into a SQL query over a table containing chronic disease indica